In [1]:
import numpy as np
import re
import jieba
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [2]:
csv_path = "C:/study/AI Training and Jobs/NLP_CV_Course/datasource-master/export_sql_1558435/sqlResult_1558435.csv"
stop_words_path = "C:/study/AI Training and Jobs/NLP_CV_Course/datasource-master/"
cleaned_data_path = "C:/study/AI Training and Jobs/NLP_CV_Course/week_7/"

In [3]:
news = pd.read_csv(csv_path, encoding='gb18030')

In [4]:
news.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [5]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 2.7+ MB


In [6]:
len(news[news['source'] == '新华社'])/len(news)

0.8778051801676133

In [7]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 2.7+ MB


In [4]:
news.dropna(subset=["content", "source"], inplace = True)

In [9]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87052 entries, 0 to 89610
Data columns (total 7 columns):
id         87052 non-null int64
author     76925 non-null object
source     87052 non-null object
content    87052 non-null object
feature    87052 non-null object
title      87052 non-null object
url        87052 non-null object
dtypes: int64(1), object(6)
memory usage: 3.3+ MB


## Cleanning up data

In [45]:
pattern = re.compile(r'\w+')
def clean_data(string):
    string = re.sub(r'\d+', 'NUM', string)
    words = re.findall(pattern, string)
    return ''.join(word for word in words)

In [46]:
with open(cleaned_data_path + 'cleaned_data.txt', 'w', encoding='utf-8') as f:
    for content in news['content']:
        cleaned_content = clean_data(np.str_(content))
        f.write(cleaned_content + "\n")

## Jieba 分词

In [47]:
def cut(string):
    return list(jieba.cut(string))

In [48]:
with open(cleaned_data_path + 'cleaned_data.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()
    
with open(stop_words_path + 'chinese_stopwords.txt', 'r', encoding='UTF-8') as f:
    stopWords = f.readlines()

In [49]:
def determine(string):
    for stopWord in stopWords:
        if stopWord == string:
            return False
    return True

In [50]:
with open(cleaned_data_path + 'jieba_data.txt', 'w', encoding='UTF-8') as f:
    for line in lines:
        cuted_words = [cutted_word for cutted_word in cut(line) if determine(cutted_word)]
        sentence = ' '.join(t for t in cuted_words)
        f.write(sentence + '\n')

#### Data imbalance Processing using choosen data

In [50]:
# with open(cleaned_data_path + 'jieba_data.txt', 'r', encoding='UTF-8') as f:
#     lines = f.readlines()
# news_pos = news[news['source'].str.contains("新华社")]
# news_neg = news[news['source'].str.contains("新华社") == 0]
# news_pos = news_pos.sample(len(news_neg))
# news_sampled = news_pos.append(news_neg)

## Generating Tfidf matrix

In [5]:
with open(cleaned_data_path + 'jieba_data.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()

In [6]:
corpus = pd.Series(lines)

In [7]:
stopWords = ['nba', 'nn', 'nnn', 'nnnum', 'nnum', 'num', 'numnum', 'numn', '一个', '一些', '一名', '一带', '一次', '一直', '一起', '一路', '万元']

In [8]:
# corpus = news['content'].apply(lambda x: np.str_(x))
print("corpus's shape is ", corpus.shape)
vectorizer = TfidfVectorizer(max_features = 350, stop_words = [])
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print("input of X's shape is ", X.shape)

corpus's shape is  (87052,)
['nba', 'nn', 'nnn', 'nnnum', 'nnum', 'num', 'numnum', '一个', '一些', '一名', '一带', '一次', '一直', '一起', '一路', '万元', '上海', '不仅', '不同', '不少', '不断', '不是', '不能', '专业', '专家', '世界', '业务', '中国', '中心', '为了', '主场', '主席', '主要', '举办', '举行', '之一', '之后', '习近平', '了解', '事件', '二线', '互联网', '交流', '产业', '产品', '人们', '人员', '人民', '人民币', '亿元', '今年', '介绍', '他们', '代表', '以上', '以及', '以来', '价格', '企业', '会议', '传统', '位于', '体育', '作为', '作品', '使用', '促进', '俄罗斯', '保护', '信息', '健康', '全国', '全球', '全面', '公司', '公开赛', '共享', '共同', '关于', '关注', '关系', '其中', '其他', '具有', '内容', '冠军', '决定', '决赛', '出现', '分别', '创新', '利用', '制造', '加强', '包括', '北京', '区域', '历史', '去年', '参与', '参加', '双方', '发展', '发布', '发现', '发生', '取得', '叙利亚', '可以', '可能', '台湾', '合作', '同时', '启动', '品牌', '因为', '国内', '国家', '国际', '图表', '地区', '地方', '城市', '基础', '基金', '增加', '增长', '外代', '大学', '夺冠', '如何', '如果', '媒体', '存在', '学习', '学校', '学生', '孩子', '安全', '完成', '实施', '实现', '宣布', '对于', '就是', '展示', '工作', '工程', '已经', '巴黎', '市场', '希望', '带来', '帮助', '平台', '广州', '庆祝', '建立', '建设',

In [9]:
train_data = X.toarray()
print("train_data's shape is {}".format(train_data.shape))

train_data's shape is (87052, 350)


## Preparing for train and test data

In [10]:
sourceMapping = {}
for index, label in enumerate(set(news['source'])):
    if label == "新华社":
        sourceMapping['新华社'] = 1
    else:
        sourceMapping[label] = 0

In [11]:
news['source'] = news['source'].map(sourceMapping)

In [12]:
test_data = news['source']
print("test_data's shape is {}".format(test_data.shape))

test_data's shape is (87052,)


In [13]:
# train_data

In [14]:
train_input, test_input, train_output, test_output = train_test_split(train_data, test_data, test_size=0.2, random_state=42)

## Model Construction with logistic regression

In [15]:
x_arr_train = train_input
x_arr_test = test_input
y_arr_train = train_output.as_matrix()
y_arr_test = test_output.as_matrix()

C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [34]:
clf = LogisticRegression(random_state=0, class_weight='balanced').fit(x_arr_train, y_arr_train)

C:\Users\WDMWDL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### 当停用词没有应用时

In [21]:
clf.score(x_arr_test, y_arr_test)

0.9755327092068232

In [22]:
y_arr_test_pred = clf.predict(x_arr_test)

In [23]:
precision_score(y_arr_test, y_arr_test_pred)

0.9814698492462312

In [24]:
recall_score(y_arr_test, y_arr_test_pred)

0.9916857070322417

In [25]:
f1_score(y_arr_test, y_arr_test_pred)

0.9865513322389191

#### 当停用词应用时 of stopWords = ['nba', 'nn', 'nnn', 'nnnum', 'nnum', 'num', 'numnum', 'numn', '一个', '一些', '一名', '一带', '一次', '一直', '一起', '一路', '万元']

In [34]:
clf.score(x_arr_test, y_arr_test)

0.9654815920969502

In [36]:
y_arr_test_pred = clf.predict(x_arr_test)

#### Precision score and recall score

In [38]:
precision_score(y_arr_test, y_arr_test_pred)

0.9754062362758015

In [39]:
recall_score(y_arr_test, y_arr_test_pred)

0.9867352119827367

In [42]:
f1_score(y_arr_test, y_arr_test_pred)

0.9810380186149236

In [46]:
y_arr_test.shape

(17411,)

In [47]:
x_arr_test.shape

(17411, 350)

#### Model with weighted balance

In [23]:
print(clf.score(x_arr_test, y_arr_test))
y_arr_test_pred = clf.predict(x_arr_test)
print(precision_score(y_arr_test, y_arr_test_pred))
print(recall_score(y_arr_test, y_arr_test_pred))
print(f1_score(y_arr_test, y_arr_test_pred))

0.9321118832921717
0.9972703698648833
0.9275196750444276
0.9611312068398554


## 判断是否为抄袭

##### 当数据为tdata[index =100]时,判断其为抄袭或不是

In [33]:
index = 100

In [34]:
val = clf.predict(x_arr_test[index].reshape(1,-1))

In [35]:
if val == y_arr_test[index]:
    print("index = {} 不是抄袭".format(index))
else:
    print("index = {} 是抄袭".format(index))

index = 100 是抄袭


## Model Construction with logistic regression

In [21]:
n_neighbors = 15
neighbors_clf = KNeighborsClassifier(n_neighbors, weights = 'distance')
neighbors_clf.fit(x_arr_train, y_arr_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=15, p=2,
           weights='distance')

#### 存储模型和导出模型

In [20]:
from sklearn.externals import joblib
import time

In [26]:
joblib.dump(neighbors_clf, 'my_model_neighbors.pkl')

['my_model_neighbors.pkl']

In [27]:
my_model_loaded = joblib.load("my_model_neighbors.pkl")

#### proformance of the k nearest neighbors 

In [46]:
start = time.clock()
y_arr_test_pred_neighbors = neighbors_clf.predict(x_arr_test[:400])
elapsed = (time.clock() - start)
print("Time used: ",elapsed)

C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Time used:  59.82750911699986


C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
precision_score(y_arr_test[:400], y_arr_test_pred_neighbors)

0.9361702127659575

In [48]:
recall_score(y_arr_test[:400], y_arr_test_pred)

0.9750692520775623

In [49]:
f1_score(y_arr_test[:400], y_arr_test_pred)

0.9552238805970149

#### 模型微调 --- 网格搜索

In [53]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'weights':['uniform', 'distance'], "n_neighbors":[2, 5, 10, 15, 20]}]
neighbors_clf_grid = KNeighborsClassifier()
grid_search = GridSearchCV(neighbors_clf_grid, param_grid, cv = 5, scoring = ['precision', 'recall', 'f1'], refit=False)

In [ ]:
grid_search.fit(x_arr_train, y_arr_train)

In [ ]:
grid_search.best_params_

In [ ]:
cvres = grid_search.cv_results_

### Model construction with Bayes

In [16]:
from sklearn.naive_bayes import GaussianNB

In [47]:
gnb = GaussianNB()
gnb.fit(x_arr_train[1:45000,:], y_arr_train[1:45000])

GaussianNB(priors=None, var_smoothing=1e-09)

In [42]:
def performance_evaluation(clf):
#     print("model's score is {}".format(clf.score(x_arr_test, y_arr_test)))
    y_arr_test_pred = clf.predict(x_arr_test)
    print("precision is {}".format(precision_score(y_arr_test, y_arr_test_pred)))
    print("recall score is {}".format(recall_score(y_arr_test, y_arr_test_pred)))
    print("f1 is {}".format(f1_score(y_arr_test, y_arr_test_pred)))

In [48]:
performance_evaluation(gnb)

model's score is 0.8464189305611395
precision is 0.9943319226118501
recall score is 0.835046966235085
f1 is 0.9077549330757555


### Model construction with random forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
clf = RandomForestClassifier(n_estimators=1,max_depth=30, random_state=0)
clf.fit(x_arr_train, y_arr_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [38]:
performance_evaluation(clf)

model's score is 0.9854689564068693
precision is 0.9911919396742919
recall score is 0.9927646610814927
f1 is 0.9919776770143006


### Model construction with SVM

In [48]:
from sklearn.svm import SVC
start = time.clock()
clf = SVC(gamma='auto', random_state=42, max_iter=500, kernel ='linear')
clf.fit(x_arr_train[1:10000], y_arr_train[1:10000])
elapse = time.clock() - start
print("consuming time is {}".format(elapse))

C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


consuming time is 14.094820975999937


C:\Users\WDMWDL\Anaconda3\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


In [49]:
performance_evaluation(clf)

precision is 0.9724523092369478
recall score is 0.9835618177202335
f1 is 0.9779755143253818
